In [1]:
import csv
import pandas as pd
import numpy as np
import networkx as nx
import sympy
import math
import matplotlib.pyplot as plt 
import knowledge_representation as KR
import itertools
import re

generating the knowledge graph that we previously built, to implement our applications on

In [2]:
G = KR.generate_graph(KR.import_equations(), KR.import_variables())
print([x for x in G.nodes()])
print([(x,y,z) for x,y,z in G.edges(data = True)])

['A', 'a', 'A_vec', 'alpha', 'B', 'beta', 'Bx', 'By', 'Bz', 'c', 'C', 'chi', 'd', 'D', 'd1', 'd2', 'delta', 'el_ct', 'E_n', 'E_den', 'Ef', 'epsilon', 'f', 'F', 'flux', 'foc', 'G', 'g', 'g_', 'gamma', 'H', 'h', 'hbar', 'H_G', 'I', 'I_0', 'I1', 'I2', 'Int', 'Int_0', 'j', 'Jz', 'K', 'k', 'k_f', 'k_G', 'k_spring', 'kappa', 'kb', 'L', 'L_rad', 'lambd', 'm', 'M', 'm_0', 'm1', 'm2', 'mob', 'mom', 'mu', 'mu_drift', 'Nn', 'n', 'n_rho', 'n_0', 'omega', 'omega_0', 'p', 'Pol', 'p_d', 'Pwr', 'pr', 'prob', 'q', 'q1', 'q2', 'r', 'R', 'r1', 'r2', 'rho', 'rho_c', 'rho_c_0', 'rho_0', 'sigma', 'sigma_den', 't', 'T', 't1', 'T1', 'T2', 'tau', 'theta', 'theta1', 'theta2', 'U', 'u', 'v', 'V', 'v1', 'V1', 'V2', 'Volt', 'w', 'x', 'x1', 'x2', 'x3', 'y', 'Y', 'y1', 'y2', 'y3', 'z', 'z1', 'z2', 'mu_S', 'L_ind', 'Z_1', 'Z_2', 'Length', 'Mass', 'Temperature', 'Voltage', 'Τime', (-3.0, 0.0, 0.0, 0.0, 0.0), (-3.0, 0.0, 1.0, 0.0, 0.0), (-2.0, 1.0, 0.0, 0.0, 1.0), (-2.0, 3.0, -1.0, 0.0, 2.0), (-2.0, 4.0, -1.0, 0.0, 2.0

Creating the list of formulas, which we use for testing

In [3]:
all_mysteries = KR.import_equations()

First perform a small scale experiment. we consider all dimensionless variables

In [4]:
print([(x,y,z) for x,y,z in G.edges(data = True) if z.get("IS_VARIABLE_IN") == True and x in
       [(x) for x,y,z in G.edges(data = True) if y== (0,0,0,0,0) and z.get("HAS_UNIT") == True]])

[('alpha', '((x1*(cos(omega*t)+alpha*cos(omega*t)**2)))', {'IS_VARIABLE_IN': True}), ('alpha', '((n*alpha/(1-(n*alpha/3))*epsilon*Ef))', {'IS_VARIABLE_IN': True}), ('alpha', '((1+n*alpha/(1-(n*alpha/3))))', {'IS_VARIABLE_IN': True}), ('alpha', '((mom*H/(kb*T)+(mom*alpha)/(epsilon*c**2*kb*T)*M))', {'IS_VARIABLE_IN': True}), ('alpha', '((2*pi*alpha/(n*d)))', {'IS_VARIABLE_IN': True}), ('alpha', '((beta*(1+alpha*cos(theta))))', {'IS_VARIABLE_IN': True}), ('beta', '((beta*(1+alpha*cos(theta))))', {'IS_VARIABLE_IN': True}), ('chi', '((sigma_den/epsilon*1/(1+chi)))', {'IS_VARIABLE_IN': True}), ('chi', '((mom*(1+chi)*B))', {'IS_VARIABLE_IN': True}), ('delta', '((I1+I2+2*sqrt(I1*I2)*cos(delta)))', {'IS_VARIABLE_IN': True}), ('g_', '((g_*q*B/(2*m)))', {'IS_VARIABLE_IN': True}), ('g_', '((g_*mom*B*Jz/(h/(2*pi))))', {'IS_VARIABLE_IN': True}), ('gamma', '((1/(gamma-1)*pr*V))', {'IS_VARIABLE_IN': True}), ('gamma', '((1/(gamma-1)*kb*v/A))', {'IS_VARIABLE_IN': True}), ('gamma', '((sqrt(gamma*pr/rho))

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[0:1], 'GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  pass


In [6]:
def generate_differentials_at_point(point, func):
    x = tf.Variable(point, dtype = float)  # Create a Tensorflow variable initialized to 1.0

    with tf.GradientTape() as t3: 
        with tf.GradientTape() as t2:
            with tf.GradientTape() as t1:
                y = func(x)

      # Compute the gradient inside the outer `t2` context manager
      # which means the gradient computation is differentiable as well.
            dy_dx = t1.gradient(y, x, unconnected_gradients=tf.UnconnectedGradients.ZERO)
        d2y_dx2 = t2.gradient(dy_dx, x, unconnected_gradients=tf.UnconnectedGradients.ZERO)
    d3y_dx3 = t3.gradient(d2y_dx2, x, unconnected_gradients=tf.UnconnectedGradients.ZERO)
    return [y.numpy(), dy_dx.numpy(), d2y_dx2.numpy(), d3y_dx3.numpy()]

def generate_taylor_series(arr):
    series = []
    for i in range(len(arr)):
        series.append(arr[i]/math.factorial(i))
    return series

# print((generate_differentials_at_point(1, tf.math.sin)))
# print(generate_taylor_series(generate_differentials_at_point(1, tf.math.sin)))

In [7]:
import automatic_differentiation as AD

def local_power(y):
    def pow(x):
        return tf.math.pow(x,y)
    return pow

point = 1

taylor_series_ground_truth = [generate_taylor_series(generate_differentials_at_point(point, tf.math.sin)),
                             generate_taylor_series(generate_differentials_at_point(point, tf.math.cos)),
                             generate_taylor_series(generate_differentials_at_point(point, tf.math.log)),
                             generate_taylor_series(generate_differentials_at_point(point, tf.math.exp)),
                             generate_taylor_series(generate_differentials_at_point(point, tf.math.square)),
                             generate_taylor_series(generate_differentials_at_point(point, tf.math.sqrt)),
                             generate_taylor_series(generate_differentials_at_point(point, local_power(3))),
                             generate_taylor_series(generate_differentials_at_point(point, tf.math.sinh)),
                             generate_taylor_series(generate_differentials_at_point(point, tf.math.cosh)),
                             generate_taylor_series(generate_differentials_at_point(point, tf.math.reciprocal))]

print(taylor_series_ground_truth)

def maape(actual, pred):
    return np.mean(np.arctan(np.abs((np.array(actual) - np.array(pred)) / np.array(pred)))) * 100

from sklearn.metrics import mean_squared_error
    

[[0.8414710164070129, 0.5403022766113281, -0.42073550820350647, -0.09005037943522136], [0.5403022766113281, -0.8414710164070129, -0.27015113830566406, 0.14024516940116882], [0.0, 1.0, -0.5, 0.3333333333333333], [2.7182817459106445, 2.7182817459106445, 1.3591408729553223, 0.4530469576517741], [1.0, 2.0, 1.0, 0.0], [1.0, 0.5, -0.125, 0.0625], [1.0, 3.0, 3.0, 1.0], [1.175201177597046, 1.5430805683135986, 0.587600588798523, 0.2571800947189331], [1.5430805683135986, 1.175201177597046, 0.7715402841567993, 0.19586686293284097], [1.0, -1.0, 1.0, -1.0]]


Testing: here the correct function is sin(x), which do the various activation functions predict? The MSE and MAAPE for the first value in the results should be the best/ lowest
    

In [8]:
#given a curve for sin(x), what does each AD nn predict?

x = np.random.rand(100000)*2+1

results = []

for i in [tf.keras.activations.sigmoid,tf.keras.activations.softplus, tf.keras.activations.tanh, tf.keras.activations.softmax]:
    results.append([np.array(AD.multiple_gradients(x, np.sin(x), point = point, activation = i, learning_rate = 0.01)).reshape(4)])
results.append(np.array(AD.three_grads(x, np.sin(x), point = point)).reshape(4))   
       


Epoch 1/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0802A
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0681
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0653
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0644
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0638
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0636
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0634
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0633
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0632
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0632
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0632
Epoch 12/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0632
Epoch 13/20
782/782 [===

In [9]:
for i in results:
    print("predicting using function")
    mse_results = []
    maape_results = []
    for j in taylor_series_ground_truth:
        mse_results.append(mean_squared_error(generate_taylor_series(np.array(i).reshape(4)),j))
        maape_results.append(maape(generate_taylor_series(np.array(i).reshape(4)),j))
    print("suggests the following with mse: ", np.array(mse_results).argsort().argsort()+1)
    print("suggests the following with maape: ", np.array(maape_results).argsort().argsort()+1)

predicting using function
suggests the following with mse:  [ 2  3  4  9  8  1 10  6  5  7]
suggests the following with maape:  [ 1  6 10  8  9  4  3  5  7  2]
predicting using function
suggests the following with mse:  [ 2  3  4  9  8  1 10  7  5  6]
suggests the following with maape:  [ 1  6 10  8  9  4  3  5  7  2]
predicting using function
suggests the following with mse:  [ 3  2  6  9  8  1 10  7  5  4]
suggests the following with maape:  [ 6  7 10  8  9  2  3  4  5  1]
predicting using function
suggests the following with mse:  [ 2  3  4  9  8  1 10  6  5  7]
suggests the following with maape:  [ 1  6 10  8  9  2  3  5  7  4]
predicting using function
suggests the following with mse:  [ 2  3  4  9  8  1 10  6  5  7]
suggests the following with maape:  [ 3  8  9  7 10  1  2  5  6  4]


/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: di

Testing. Here, the correct function is cos(x)

In [10]:
#given a curve for cos(x), what does each AD nn predict?

x = np.random.rand(100000)*2+1
results = []

for i in [tf.keras.activations.sigmoid,tf.keras.activations.softplus, tf.keras.activations.tanh, tf.keras.activations.softmax]:
    results.append([np.array(AD.multiple_gradients(x, np.cos(x), point = point, activation = i, learning_rate = 0.01)).reshape(4)])
results.append(np.array(AD.three_grads(x, np.cos(x), point = point)).reshape(4))   
    
for i in results:
    print("predicting using function")
    mse_results = []
    maape_results = []
    for j in taylor_series_ground_truth:
        mse_results.append(mean_squared_error(generate_taylor_series(np.array(i).reshape(4)),j))
        maape_results.append(maape(generate_taylor_series(np.array(i).reshape(4)),j))
    print("suggests the following with mse: ", np.array(mse_results).argsort().argsort()+1)
    print("suggests the following with maape: ", np.array(maape_results).argsort().argsort()+1)

Epoch 1/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2447
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2322
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2298
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2287
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2280
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2278
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2273
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2269
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2267
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2264
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2256
Epoch 12/20
782/782 [==============================] - 1s 2ms/step - loss: 0.2254
Epoch 13/20
782/782 [====

/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: di

In [11]:
#given a curve for ln(x), what does each AD nn predict?

x = np.random.rand(100000)*2+1
results = []

for i in [tf.keras.activations.sigmoid,tf.keras.activations.softplus, tf.keras.activations.tanh, tf.keras.activations.softmax]:
    results.append([np.array(AD.multiple_gradients(x, np.log(x), point = point, activation = i, learning_rate = 0.01)).reshape(4)])
results.append(np.array(AD.three_grads(x, np.log(x), point = point)).reshape(4))   
    
for i in results:
    print("predicting using function")
    mse_results = []
    maape_results = []
    for j in taylor_series_ground_truth:
        mse_results.append(mean_squared_error(generate_taylor_series(np.array(i).reshape(4)),j))
        maape_results.append(maape(generate_taylor_series(np.array(i).reshape(4)),j))
    print("suggests the following with mse: ", np.array(mse_results).argsort().argsort()+1)
    print("suggests the following with maape: ", np.array(maape_results).argsort().argsort()+1)

Epoch 1/20
782/782 [==============================] - 1s 2ms/step - loss: 0.1090
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0988
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0963
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0954
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0949
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0948
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0947
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0947
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0946
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0944
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0944
Epoch 12/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0944
Epoch 13/20
782/782 [====

/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: di

In [12]:
#given a curve for exp(x), what does each AD nn predict?

x = np.random.rand(100000)*2+1
results = []

for i in [tf.keras.activations.sigmoid,tf.keras.activations.softplus, tf.keras.activations.tanh, tf.keras.activations.softmax]:
    results.append([np.array(AD.multiple_gradients(x, np.exp(x), point = point, activation = i, learning_rate = 0.01)).reshape(4)])
results.append(np.array(AD.three_grads(x, np.exp(x), point = point)).reshape(4))   
    
for i in results:
    print("predicting using function")
    mse_results = []
    maape_results = []
    for j in taylor_series_ground_truth:
        mse_results.append(mean_squared_error(generate_taylor_series(np.array(i).reshape(4)),j))
        maape_results.append(maape(generate_taylor_series(np.array(i).reshape(4)),j))
    print("suggests the following with mse: ", np.array(mse_results).argsort().argsort()+1)
    print("suggests the following with maape: ", np.array(maape_results).argsort().argsort()+1)

Epoch 1/20
782/782 [==============================] - 1s 2ms/step - loss: 22.1454
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 7.5894
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 3.9469
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 2.9725
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 2.5566
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 2.3406
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 2.1464
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 2.0099
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 1.9185
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 1.8153
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 1.7414
Epoch 12/20
782/782 [==============================] - 1s 2ms/step - loss: 1.6744
Epoch 13/20
782/782 [===

782/782 [==============================] - 1s 2ms/step - loss: 0.8788
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 0.5559
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 0.5171
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 0.5017
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 0.4889
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 0.4850
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 0.4801
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 0.4739
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 0.4702
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 0.4662
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 0.4624
Epoch 12/20
782/782 [==============================] - 1s 2ms/step - loss: 0.4568
Epoch 13/20
782/782 [===============

/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: di

In [13]:
#given a curve for square(x), what does each AD nn predict?

x = np.random.rand(100000)*2+1
results = []

for i in [tf.keras.activations.sigmoid,tf.keras.activations.softplus, tf.keras.activations.tanh, tf.keras.activations.softmax]:
    results.append([np.array(AD.multiple_gradients(x, x**2, point = point, activation = i, learning_rate = 0.01)).reshape(4)])
results.append(np.array(AD.three_grads(x, x**2, point = point)).reshape(4))   
    
for i in results:
    print("predicting using function")
    mse_results = []
    maape_results = []
    for j in taylor_series_ground_truth:
        mse_results.append(mean_squared_error(generate_taylor_series(np.array(i).reshape(4)),j))
        maape_results.append(maape(generate_taylor_series(np.array(i).reshape(4)),j))
    print("suggests the following with mse: ", np.array(mse_results).argsort().argsort()+1)
    print("suggests the following with maape: ", np.array(maape_results).argsort().argsort()+1)

Epoch 1/20
782/782 [==============================] - 1s 2ms/step - loss: 5.4627
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 5.3031
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 4.3970
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 2.4343
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 1.5743
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 1.2282
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 1.0260
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 0.9016
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 0.8130
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 0.7471
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 0.7028
Epoch 12/20
782/782 [==============================] - 1s 2ms/step - loss: 0.6626
Epoch 13/20
782/782 [====

/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: di

In [14]:
#given a curve for sqrt(x), what does each AD nn predict?

x = np.random.rand(100000)*2+1
results = []

for i in [tf.keras.activations.sigmoid,tf.keras.activations.softplus, tf.keras.activations.tanh, tf.keras.activations.softmax]:
    results.append([np.array(AD.multiple_gradients(x, np.sqrt(x), point = point, activation = i, learning_rate = 0.01)).reshape(4)])
results.append(np.array(AD.three_grads(x, np.sqrt(x), point = point)).reshape(4))   
    
for i in results:
    print("predicting using function")
    mse_results = []
    maape_results = []
    for j in taylor_series_ground_truth:
        mse_results.append(mean_squared_error(generate_taylor_series(np.array(i).reshape(4)),j))
        maape_results.append(maape(generate_taylor_series(np.array(i).reshape(4)),j))
    print("suggests the following with mse: ", np.array(mse_results).argsort().argsort()+1)
    print("suggests the following with maape: ", np.array(maape_results).argsort().argsort()+1)

Epoch 1/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0641
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0481
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0452
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0443
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0438
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0436
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0435A:
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0434
Epoch 9/20
782/782 [==============================] - 1s 1ms/step - loss: 0.0434
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0434
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0433
Epoch 12/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0433
Epoch 13/20
782/782 [==

/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: di

In [15]:
#given a curve for cube(x), what does each AD nn predict?

x = np.random.rand(100000)*2+1
results = []

for i in [tf.keras.activations.sigmoid,tf.keras.activations.softplus, tf.keras.activations.tanh, tf.keras.activations.softmax]:
    results.append([np.array(AD.multiple_gradients(x, x**3, point = point, activation = i, learning_rate = 0.01)).reshape(4)])
results.append(np.array(AD.three_grads(x, x**3, point = point)).reshape(4))   
    
for i in results:
    print("predicting using function")
    mse_results = []
    maape_results = []
    for j in taylor_series_ground_truth:
        mse_results.append(mean_squared_error(generate_taylor_series(np.array(i).reshape(4)),j))
        maape_results.append(maape(generate_taylor_series(np.array(i).reshape(4)),j))
    print("suggests the following with mse: ", np.array(mse_results).argsort().argsort()+1)
    print("suggests the following with maape: ", np.array(maape_results).argsort().argsort()+1)

Epoch 1/20
782/782 [==============================] - 1s 2ms/step - loss: 39.1119
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 8.1059
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 5.5396
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 4.5986
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 4.0640
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 3.6973
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 3.3796
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 3.1809
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 3.0267
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 2.8347
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 2.7584
Epoch 12/20
782/782 [==============================] - 1s 2ms/step - loss: 2.6166
Epoch 13/20
782/782 [===

/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: di

In [16]:
x = np.random.rand(100000)*2+1
results = []

for i in [tf.keras.activations.sigmoid,tf.keras.activations.softplus, tf.keras.activations.tanh, tf.keras.activations.softmax]:
    results.append([np.array(AD.multiple_gradients(x, np.sinh(x), point = point, activation = i, learning_rate = 0.01)).reshape(4)])
results.append(np.array(AD.three_grads(x, np.sinh(x), point = point)).reshape(4))   
    
for i in results:
    print("predicting using function")
    mse_results = []
    maape_results = []
    for j in taylor_series_ground_truth:
        mse_results.append(mean_squared_error(generate_taylor_series(np.array(i).reshape(4)),j))
        maape_results.append(maape(generate_taylor_series(np.array(i).reshape(4)),j))
    print("suggests the following with mse: ", np.array(mse_results).argsort().argsort()+1)
    print("suggests the following with maape: ", np.array(maape_results).argsort().argsort()+1)

Epoch 1/20
782/782 [==============================] - 1s 2ms/step - loss: 6.0691
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 5.8788
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 4.6874
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 2.8374
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 1.9996
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 1.5320
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 1.2478
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 1.0785
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 0.9711
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 0.8900
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 0.8389
Epoch 12/20
782/782 [==============================] - 1s 2ms/step - loss: 0.7938
Epoch 13/20
782/782 [====

/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: di

In [17]:
x = np.random.rand(100000)*2+1
results = []

for i in [tf.keras.activations.sigmoid,tf.keras.activations.softplus, tf.keras.activations.tanh, tf.keras.activations.softmax]:
    results.append([np.array(AD.multiple_gradients(x, np.cosh(x), point = point, activation = i, learning_rate = 0.01)).reshape(4)])
results.append(np.array(AD.three_grads(x, np.cosh(x), point = point)).reshape(4))   
    
for i in results:
    print("predicting using function")
    mse_results = []
    maape_results = []
    for j in taylor_series_ground_truth:
        mse_results.append(mean_squared_error(generate_taylor_series(np.array(i).reshape(4)),j))
        maape_results.append(maape(generate_taylor_series(np.array(i).reshape(4)),j))
    print("suggests the following with mse: ", np.array(mse_results).argsort().argsort()+1)
    print("suggests the following with maape: ", np.array(maape_results).argsort().argsort()+1)

Epoch 1/20
782/782 [==============================] - 1s 2ms/step - loss: 5.7596
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 5.5953
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 4.6545
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 2.7625
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 1.8778
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 1.4488
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 1.1881
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 1.0417
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 0.9428
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 0.8624
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 0.8072
Epoch 12/20
782/782 [==============================] - 1s 2ms/step - loss: 0.7719
Epoch 13/20
782/782 [====

/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: di

In [18]:
x = np.random.rand(100000)*2+1
results = []

for i in [tf.keras.activations.sigmoid,tf.keras.activations.softplus, tf.keras.activations.tanh, tf.keras.activations.softmax]:
    results.append([np.array(AD.multiple_gradients(x, 1/x, point = point, activation = i, learning_rate = 0.01)).reshape(4)])
results.append(np.array(AD.three_grads(x, 1/x, point = point)).reshape(4))   
    
for i in results:
    print("predicting using function")
    mse_results = []
    maape_results = []
    for j in taylor_series_ground_truth:
        mse_results.append(mean_squared_error(generate_taylor_series(np.array(i).reshape(4)),j))
        maape_results.append(maape(generate_taylor_series(np.array(i).reshape(4)),j))
    print("suggests the following with mse: ", np.array(mse_results).argsort().argsort()+1)
    print("suggests the following with maape: ", np.array(maape_results).argsort().argsort()+1)

Epoch 1/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0437
Epoch 2/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0352
Epoch 3/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0332
Epoch 4/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0325
Epoch 5/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0321
Epoch 6/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0320
Epoch 7/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0318
Epoch 8/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0317
Epoch 9/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0316
Epoch 10/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0316
Epoch 11/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0316
Epoch 12/20
782/782 [==============================] - 1s 2ms/step - loss: 0.0315
Epoch 13/20
782/782 [====

/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/ziyu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: di